In [9]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import re
import spacy
import itertools
from bs4 import BeautifulSoup

In [10]:
with open('julia.md','r') as f:
    all_text = f.readlines()

FileNotFoundError: [Errno 2] No such file or directory: 'julia.md'

In [ ]:
def clean_text(text):
    

['![](assets_j/img-0147.jpg)\n',
 '\n',
 '![](assets_j/img-0148.jpg)\n',
 '\n',
 'Contents lists available at [ScienceDirect](http://www.ScienceDirect.com/) Journal of Computational Physics\n',
 '\n',
 '### www.elsevier.com/locate/jcp\n',
 '\n',
 'A fast marching algorithm for the factored eikonal equation Eran Treister a,∗ Eldad Haber a,b  \n',
 'a *Department of Earth and Ocean Sciences, The University of British Columbia, Vancouver, BC, Canada*  \n',
 'b *Department of Mathematics, The University of British Columbia, Vancouver, BC, Canada*\n',
 '\n',
 '![](assets_j/img-0149.jpg)\n',
 '\n',
 '### a r t i c l e i n f o\n',
 '\n',
 '*Article history:*  \n',
 'Received 14 April 2016  \n',
 'Received in revised form 5 August 2016 Accepted 8 August 2016  \n',
 'Available online 12 August 2016 *Keywords:*  \n',
 'Eikonal equation  \n',
 'Factored eikonal equation  \n',
 'Fast Marching  \n',
 'First arrival  \n',
 'Travel time tomography  \n',
 'Gauss–Newton  \n',
 'Seismic imaging\n',
 '\n

In [ ]:
nlp = spacy.load("en_core_web_sm")
all_text = open('julia.md','r').read()
all_doc = nlp(all_text)
print([token.text for token in all_doc])

['!', '[', ']', '(', 'assets_j', '/', 'img-0147.jpg', ')', '\n\n', '!', '[', ']', '(', 'assets_j', '/', 'img-0148.jpg', ')', '\n\n', 'Contents', 'lists', 'available', 'at', '[', 'ScienceDirect](http://www', '.', 'ScienceDirect.com/', ')', 'Journal', 'of', 'Computational', 'Physics', '\n\n', '#', '#', '#', 'www.elsevier.com/locate/jcp', '\n\n', 'A', 'fast', 'marching', 'algorithm', 'for', 'the', 'factored', 'eikonal', 'equation', 'Eran', 'Treister', 'a,∗', 'Eldad', 'Haber', 'a', ',', 'b', ' \n', 'a', '*', 'Department', 'of', 'Earth', 'and', 'Ocean', 'Sciences', ',', 'The', 'University', 'of', 'British', 'Columbia', ',', 'Vancouver', ',', 'BC', ',', 'Canada', '*', ' \n', 'b', '*', 'Department', 'of', 'Mathematics', ',', 'The', 'University', 'of', 'British', 'Columbia', ',', 'Vancouver', ',', 'BC', ',', 'Canada', '*', '\n\n', '!', '[', ']', '(', 'assets_j', '/', 'img-0149.jpg', ')', '\n\n', '#', '#', '#', 'a', 'r', 't', 'i', 'c', 'l', 'e', 'i', 'n', 'f', 'o', '\n\n', '*', 'Article', 'hist

In [ ]:
def elem_to_text(elem, default=''):
    if elem:
        return elem.getText()
    else:
        return default

## Regex parsing with bs4

In [18]:
def read_tei(tei_file):
    with open(tei_file, 'r') as tei:
        soup = BeautifulSoup(tei, 'lxml')
        return soup

def search(text,find,n):
    '''Searches for text, and retrieves n words either side of the text, which are retuned seperatly'''
    word = r"\W*([\w]+)"
    try:
        groups = re.search(r'{}\W*{}{}'.format(word*n,find,word*n), text).groups()
        return groups[:n],groups[n:]
    except:
        return "Not found"

In [20]:
soup = read_tei('./case-studies/arxiv-corpus/mine5/parsed_xml/1606.04671.tei.xml')

In [21]:
soup.title.getText()

'Progressive Neural Networks'

In [ ]:
soup.p.getText(separator=' ', strip=True)

'Learning to solve complex sequences of tasks-while both leveraging transfer and avoiding catastrophic forgetting-remains a key obstacle to achieving human-level intelligence. The progressive networks approach represents a step forward in this direction: they are immune to forgetting and can leverage prior knowledge via lateral connections to previously learned features. We evaluate this architecture extensively on a wide variety of reinforcement learning tasks (Atari and 3D maze games), and show that it outperforms common baselines based on pretraining and finetuning. Using a novel sensitivity measure, we demonstrate that transfer occurs at both low-level sensory and high-level control layers of the learned policy.'

In [ ]:
idno_elem = soup.find('idno', type='DOI')
print(f"The DOI is {idno_elem.getText()}")

AttributeError: 'NoneType' object has no attribute 'getText'

In [22]:
paras = [t.getText(separator=' ', strip=True) for t in soup.find_all('p')]

In [23]:
emails = [t.getText(separator=' ', strip=True) for t in soup.find_all('email')]

In [25]:
paras

['Learning to solve complex sequences of tasks-while both leveraging transfer and avoiding catastrophic forgetting-remains a key obstacle to achieving human-level intelligence. The progressive networks approach represents a step forward in this direction: they are immune to forgetting and can leverage prior knowledge via lateral connections to previously learned features. We evaluate this architecture extensively on a wide variety of reinforcement learning tasks (Atari and 3D maze games), and show that it outperforms common baselines based on pretraining and finetuning. Using a novel sensitivity measure, we demonstrate that transfer occurs at both low-level sensory and high-level control layers of the learned policy.',
 'Finetuning remains the method of choice for transfer learning with neural networks: a model is pretrained on a source domain (where data is often abundant), the output layers of the model are adapted to the target domain, and the network is finetuned via backpropagatio

In [ ]:
l = []
count = 0
for i in range(len(paras)):
    find = search(paras[i], 'Problem', 1)
    if (find!="Not found"):
        l.append(find)
        count+=1

## Manual labels

In [140]:
from pandas import read_csv

In [144]:
max_articles=50
path_corpus = './case-studies/arxiv-corpus/mine50/'
scrape_df = read_csv(path_corpus + 'scrape_df_' + str(max_articles)+ '.csv')

# TODO index label for scrape_df
# scrape_df['id'][0:50].to_csv(path_corpus + 'scrape_id.csv')
scrape_df[['id']].to_csv('scrape_id.csv',index_label="index")

In [138]:
manual_eval =  read_csv(path_corpus + 'manual_eval.csv')
gunderson_vars = ["id","problem","objective","research_method","research_questions","pseudocode","training_data","validation_data","test_data","results","hypothesis","prediction","method_source_code","hardware_specifications","software_dependencies","experiment_setup","experiment_source_code","research_type","affiliation"]

print(manual_eval.iloc[0])

FileNotFoundError: [Errno 2] No such file or directory: './case-studies/arxiv-corpus/mine50/manual_eval.csv'

## Flashtext

In [29]:
from flashtext import KeywordProcessor
import exrex

In [63]:
keys_problem = list(exrex.generate('((((P|p)roblem) ((S|s)tatement)))|((((R|r)esearch) ((P|p)roblem)))|((P|p)roblems?)'))
keys_objective = list(exrex.generate('((((O|o)bjective)))|((((G|g)oal)|((((R|r)esearch) ((O|o)bjective))))|((((R|r)esearch) ((G|g)oal))))'))
keys_research_method = list(exrex.generate('((R|r)esearch (M|m)ethods?)|'\
    '(M|m)ethods?'))
keys_research_questions = list(exrex.generate('((((R|r)esearch)))|((((Q|q)uestions?)|'\
    '((((R|r)esearch) ((O|o)bjective))))|'\
    '((((R|r)esearch) ((G|g)oal))))'))
keys_pseudocode = list(exrex.generate('((P|p)seudo-?code)|(Algorithm [1-9])'))
keys_experiment_setup = list(exrex.generate('(E|e)xperimental( |-)(S|s)etup|'\
                                            '((H|h)yper( |-)(P|p)arameters)'))
keys_hypothesis = list(exrex.generate('(H|h)ypothes(i|e)s'))
keys_prediction = list(exrex.generate('(P|p)redictions?'))
keys_hardware_specifications = list(exrex.generate('(H|h)ardware (S|s)pecification(s)'))
keys_software_dependencies = list(exrex.generate('(S|s)oftware (D|d)ependencies'))
keys_affiliation = list(exrex.generate('@edu'))
keys_method_source_code = list(exrex.generate('(G|g)it( )?(H|h)ub|'\
    '(B|b)it(B|b)ucket|'\
        '(G|g)it( )?(L|l)ab'
))
keys_training_data = list(exrex.generate('(T|t)raining (D|d)ata'))
keys_validation_data = list(exrex.generate('(V|v)alidation (D|d)ata'))
keys_test_data = list(exrex.generate('(T|t)est (D|d)ata'))

keys_test_data

['Test Data', 'Test data', 'test Data', 'test data']

In [ ]:
def caps_combination(input_str):
    words = input_str.split(" ")
    result = []
    for word in words:
        print(word)
        # result.append(list(map(''.join, itertools.product(*((c.lower(), c.upper()) for c in word[0])))))
        # result.append(word[0].upper)
        result.append(word.lower())
        result.append(word.title())
    return result
st ="abc adc"
print(caps_combination(st))

abc
adc
['abc', 'Abc', 'adc', 'Adc']


In [132]:
keyword_processor = KeywordProcessor(case_sensitive=True)
keyword_dict = {
    "problem": keys_problem,
    "objective": keys_objective,
    "research_method": keys_research_method,
    "research_questions": keys_research_questions,
    "pseudocode": keys_pseudocode,
    
    "hypothesis": keys_hypothesis,
    "prediction": keys_prediction,
    "method_source_code": keys_method_source_code,
    "hardware_specifications": keys_hardware_specifications,
    "software_dependencies": keys_software_dependencies,
    "experiment_setup": keys_experiment_setup,
    
    "affiliation": keys_affiliation,
}
keyword_processor.add_keywords_from_dict(keyword_dict)

all_found_paras = []
edu_emails = []

for i in range(len(paras)):
    all_found_paras.append(keyword_processor.extract_keywords(paras[i], span_info=True))
for i in range(len(emails)):
    edu_emails.append(keyword_processor.extract_keywords(emails[i], span_info=True))
    
non_empty_found_paras = [x for x in all_found_paras if x != []]
non_empty_found_emails = [x for x in edu_emails if x != []]
edu_emails, non_empty_found_emails

([[], [], [], [], []], [])

In [32]:
for found_word in non_empty_found_paras[0]:
    print(found_word)
    print(paras[1][:found_word[1]] + "***____" + paras[1][found_word[1]:found_word[2]] + "___***" + paras[1][found_word[2]:])

('research_method', 23, 29)
Finetuning remains the ***____method___*** of choice for transfer learning with neural networks: a model is pretrained on a source domain (where data is often abundant), the output layers of the model are adapted to the target domain, and the network is finetuned via backpropagation. This approach was pioneered in [7] by transferring knowledge from a generative to a discriminative model, and has since been generalized with great success [11] . Unfortunately, the approach has drawbacks which make it unsuitable for transferring across multiple tasks: if we wish to leverage knowledge acquired over a sequence of experiences, which model should we use to initialize subsequent models? This seems to require not only a learning method that can support transfer learning without catastrophic forgetting, but also foreknowledge of task similarity. Furthermore, while finetuning may allow us to recover expert performance in the target domain, it is a destructive process w

In [131]:
print(all_found_emails)
print(emails)
for found_email in non_empty_found_emails:
    print(found_email)

[[], [], [], [], []]
['andreirusu@google.com', 'gdesjardins@google.com', 'soyer@google.com', 'kirkpatrick@google.com', 'korayk@google.com']


In [40]:
print(non_empty_found_paras)

[[('research_method', 23, 29), ('research_method', 717, 723), ('problem', 2271, 2279)], [('objective', 38, 42)], [('research_method', 207, 214), ('research_method', 239, 245), ('research_method', 304, 310)], [('prediction', 345, 355), ('research_method', 422, 428), ('research_method', 479, 485)], [('research_questions', 153, 161), ('research_method', 182, 189), ('research_method', 346, 353)], [('experiment_setup', 1037, 1055)], [('hypothesis', 1129, 1139)], [('problem', 223, 230)], [('research_questions', 104, 112)], [('hypothesis', 180, 190), ('problem', 503, 510)], [('research_method', 24, 31), ('research_method', 223, 229), ('research_method', 380, 386)], [('research_method', 371, 377), ('research_method', 394, 400)], [('experiment_setup', 22, 38)]]


In [89]:
max_articles=50
path_corpus = './case-studies/arxiv-corpus/mine50v2/'

manual_eval =  read_csv(path_corpus + 'manual_eval.csv')
gunderson_vars = ["id","problem","objective","research_method","research_questions","pseudocode","training_data","validation_data","test_data","results","hypothesis","prediction","method_source_code","hardware_specifications","software_dependencies","experiment_setup","experiment_source_code","research_type","affiliation"]

# print(manual_eval.iloc[0])
repro_eval = read_csv(path_corpus + 'scrape_df_' + str(max_articles)+ '.csv')

# TODO index label for repro_eval
# repro_eval['id'][0:50].to_csv(path_corpus + 'scrape_id.csv')
repro_eval = repro_eval[['id']]
repro_eval = repro_eval.reindex(gunderson_vars, axis=1) 

In [104]:
repro_eval[0]

,id,problem,objective,research_method,research_questions,pseudocode,training_data,validation_data,test_data,results,hypothesis,prediction,method_source_code,hardware_specifications,software_dependencies,experiment_setup,experiment_source_code,research_type,affiliation
0,1606.04671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1903.09668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1904.10554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1908.05659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1909.00931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1911.03867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2002.05905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2004.05258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2005.02607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2007.09855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
found_vars = set()
for i in non_empty_found_paras:
    for j in i:
        found_vars.add(j[0])

for i in repro_eval.columns[2:]:
    if i in found_vars:
        repro_eval[i][0] = 1
    else:
        repro_eval[i][0] = 0
[manual_eval.iloc[0][2:].values, repro_eval.iloc[0][2:].values]

[array([1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
        1.]),
 array([1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0.])]

In [130]:
pd.DataFrame([manual_eval.iloc[0][2:].values, repro_eval.iloc[0][2:].values], columns=gunderson_vars[1:], index=['manual_eval','repro_eval'])

,problem,objective,research_method,research_questions,pseudocode,training_data,validation_data,test_data,results,hypothesis,prediction,method_source_code,hardware_specifications,software_dependencies,experiment_setup,experiment_source_code,research_type,affiliation
manual_eval,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
repro_eval,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN


In [120]:
manual_eval.drop(['index'])

,index,id,problem,objective,research_method,research_questions,pseudocode,training_data,validation_data,test_data,results,hypothesis,prediction,method_source_code,hardware_specifications,software_dependencies,experiment_setup,experiment_source_code,research_type,affiliation
0,0,1606.04671,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1,1903.09668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1904.10554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,1908.05659,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,1909.00931,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,5,1911.03867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,6,2002.05905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,7,2004.05258,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,8,2005.02607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,9,2007.09855,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
